# Balanced network of excitatory and inhibitory neurons.

An implementation of benchmarks from Brette et al. (2007) Journal of Computational Neuroscience 23: 349-398
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2638500/ (see Fig. 24). 

In the paper, they compared simulations from NEST and NEURON. Here, we are going to simulate the balanced network with SpiNNaker.

The network is based on the COBA models of Vogels & Abbott (J. Neurosci, 2005) https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6725859/ .

You can look also the explicit model in the supplementary material of Brette et al. (2007) |https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2638500/#APP2

<img src="../../extra/VA_EXC_SPIKES.png">



In [1]:
pwd

'/home/bbpnrsoa/CNT-2025/notebooks/intro'

# use spinnaker and brain

In [2]:
# %matplotlib inline   ← solo se sei in Jupyter

# PyNN backend: preferisce SpiNNaker, fallback su Brian2
try:
    import pyNN.spiNNaker as sim
except ModuleNotFoundError:
    import pyNN.brian2 as sim

from pyNN import space
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
import datetime
import json
import warnings
from fooof import FOOOF
from scipy.signal import welch
from matplotlib import colors
import os
import datetime

# Utilità personalizzate
import python_utils as pu

# Disattiva warning inutili
warnings.filterwarnings('ignore')

# Parametri di salvataggio
fileName = 'VA_balance-network'
savePath = '../outputs/'  # Assicurati che esista o usa os.makedirs()
dt_string = datetime.datetime.today().strftime('%Y%m%d_%H%M%S')  # formato sicuro per filesystem
tag = dt_string
saveName = f'{savePath}{fileName}-{tag}'
os.makedirs(saveName, exist_ok=True)
print(f'Saving in: {saveName}')

# Dizionario per i parametri
PARS = {}


<ipython-input-2-12b45f7115f3>:17: DeprecationWarning: 
The `fooof` package is being deprecated and replaced by the `specparam` (spectral parameterization) package.
This version of `fooof` (1.1) is fully functional, but will not be further updated.
New projects are recommended to update to using `specparam` (see Changelog for details).
  from fooof import FOOOF


Saving in: ../outputs/VA_balance-network-20250602_114444


# step0: import the simulator

In [3]:
import socket
try:
    import pyNN.spiNNaker as sim
except ModuleNotFoundError:
    import pyNN.brian2 as sim
    
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time
from pyNN import space 

# step1: setup the simulator

In [4]:
dt = 1          # (ms) simulation timestep
tstop = 1000      # (ms) simulaton duration
delay = 2 # (ms) 

sim.setup(
    timestep=dt, 
    min_delay=delay, 
    max_delay=delay) # [ms] # not that the max_delay supported by SpiNNaker is timestep * 144

rngseed = 98766987
rng = NumpyRNG(seed=rngseed, parallel_safe=True)

2025-06-02 11:44:44 INFO: Read cfg files: /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2025-06-02 11:44:44 INFO: Will search these locations for binaries: /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/common_model_binaries : /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/model_binaries
2025-06-02 11:44:44 WARNING: /home/bbpnrsoa/CNT-2025/notebooks/intro/reports has 11 old reports that have not been closed
2025-06-02 11:44:44 INFO: Setting time scale factor to 10.
2025-06-02 11:44:44 INFO: Setting machine time step to 1000 micro-seconds.


['/home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


# step2: decide the cell types and parameters

In [5]:
# cell type
celltype = sim.IF_cond_exp
#  Leaky integrate and fire model with fixed threshold and exponentially-decaying post-synaptic conductance.
#  http://neuralensemble.org/docs/PyNN/_modules/pyNN/standardmodels/cells.html#IF_cond_exp
    
# Cell parameters
area = 20000.     # (µm²)
tau_m = 20.       # (ms)
cm = 1.           # (µF/cm²)
g_leak = 5e-5     # (S/cm²)

E_leak = None
E_leak = -60.  # (mV)
v_thresh = -50.   # (mV)
v_reset = -60.    # (mV)
t_refrac = 5.     # (ms) (clamped at v_reset)
v_mean = -60.     # (mV) mean membrane potential, for calculating CUBA weights
tau_exc = 5.      # (ms)
tau_inh = 10.     # (ms)
Erev_exc = 0.     # (mV)
Erev_inh = -80.   # (mV)

# === Calculate derived parameters ===
area = area * 1e-8                     # convert to cm²
cm = cm * area * 1000                  # convert to nF
Rm = 1e-6 / (g_leak * area)            # membrane resistance in MΩ
assert tau_m == cm * Rm                # just to check

cell_params = {'tau_m': tau_m,
               'tau_syn_E': tau_exc,
               'tau_syn_I': tau_inh,
               'v_rest': E_leak,
               'v_reset': v_reset,
               'v_thresh': v_thresh,
               'cm': cm,
               'tau_refrac': t_refrac,
               'i_offset': 0
               }

cell_params['e_rev_E'] = Erev_exc
cell_params['e_rev_I'] = Erev_inh

print(cell_params)

{'tau_m': 20.0, 'tau_syn_E': 5.0, 'tau_syn_I': 10.0, 'v_rest': -60.0, 'v_reset': -60.0, 'v_thresh': -50.0, 'cm': 0.2, 'tau_refrac': 5.0, 'i_offset': 0, 'e_rev_E': 0.0, 'e_rev_I': -80.0}


# step3: making cell populations


In [6]:
n = 1500       # number of cells
r_ei = 4.0        # number of excitatory cells:number of inhibitory cells
n_exc = int(round((n * r_ei / (1 + r_ei))))  # number of excitatory cells
n_inh = n - n_exc                            # number of inhibitory cells
print("{} {}".format(n_exc, n_inh))

pops = {
    'exc': sim.Population(
                                n_exc, 
                                celltype(**cell_params), 
                                label="Excitatory_Cells"),

    'inh': sim.Population(
                                n_inh, 
                                celltype(**cell_params), 
                                label="Inhibitory_Cells")
}

pops['exc'].record(["spikes", 'v', 'gsyn_exc', 'gsyn_inh'])
pops['inh'].record(["spikes", 'v', 'gsyn_exc', 'gsyn_inh'])


uniformDistr = RandomDistribution('uniform', [v_reset, v_thresh], rng=rng)
pops['exc'].initialize(v=uniformDistr)
pops['inh'].initialize(v=uniformDistr)

#sim.set_number_of_neurons_per_core(sim.IF_cond_exp, 50)      # this will set
# 50 neurons per core
pops.keys()

1200 300


dict_keys(['exc', 'inh'])

# step4: define the synapse types

In [7]:
# Synapse parameters
Gexc = None
Ginh = None
Gexc = 4.     # (nS) #1
Ginh = 51.    # (nS) #1
w_exc = Gexc * 1e-3              # We convert conductances to uS
w_inh = Ginh * 1e-3

exc_synapses = sim.StaticSynapse(weight=w_exc, delay=delay)
inh_synapses = sim.StaticSynapse(weight=w_inh, delay=delay)

# step5: select the connections algorithm

In [8]:
pconn = 0.02      # connection probability (2%)

exc_conn = sim.FixedProbabilityConnector(pconn, rng=rng)
inh_conn = sim.FixedProbabilityConnector(pconn, rng=rng)

# step6: make the projections

In [9]:
connections = {
    
    'e2e': sim.Projection(
        pops['exc'],
        pops['exc'], 
        exc_conn, 
        receptor_type='excitatory',
        synapse_type=exc_synapses),
    
    'e2i': sim.Projection(
        pops['exc'], 
        pops['inh'], 
        exc_conn, 
        receptor_type='excitatory',
        synapse_type=exc_synapses),
    
    'i2e': sim.Projection(
        pops['inh'], 
        pops['exc'], 
        inh_conn, 
        receptor_type='inhibitory',
        synapse_type=inh_synapses),
    
    'i2i': sim.Projection(
        pops['inh'],
        pops['inh'],
        inh_conn, 
        receptor_type='inhibitory',
        synapse_type=inh_synapses)
    
        }

connections.keys()

dict_keys(['e2e', 'e2i', 'i2e', 'i2i'])

# step7: setting the thalamic stimulus

In [10]:
n_thalamic_cells = 20 
stim_dur = 50.    # (ms) duration of random stimulation
rate = 100.       # (Hz) frequency of the random stimulation

exc_conn = None

pops['thalamus'] = sim.Population(
    n_thalamic_cells, 
    sim.SpikeSourcePoisson(rate=rate, duration=stim_dur),
    label="expoisson")
pops['thalamus'].record("spikes")


rconn = 0.01
ext_conn = sim.FixedProbabilityConnector(rconn)

connections['ext2e'] = sim.Projection(
    pops['thalamus'], 
    pops['exc'], 
    ext_conn, 
    receptor_type='excitatory',
    synapse_type=sim.StaticSynapse(weight=0.1))

connections['ext2i'] = sim.Projection(
    pops['thalamus'], 
    pops['inh'], 
    ext_conn, 
    receptor_type='excitatory',
    synapse_type=sim.StaticSynapse(weight=0.1))

connections.keys(), pops.keys()

(dict_keys(['e2e', 'e2i', 'i2e', 'i2i', 'ext2e', 'ext2i']),
 dict_keys(['exc', 'inh', 'thalamus']))

# step8: run simulation

In [11]:
tstop = 1000 # (ms)
# simulation run

tic = time.time()
sim.run(tstop)
toc = time.time() - tic

2025-06-02 11:44:44 INFO: Starting execution process
2025-06-02 11:44:44 INFO: Simulating for 1000 1.0ms timesteps using a hardware timestep of 10000us
Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2025-06-02 11:44:45 INFO: Time 0:00:00.012710 taken by SpynnakerSplitterSelector
Adding delay extensions as required
|0%                          50%                         100%|
2025-06-02 11:44:45 INFO: Time 0:00:00.020840 taken by DelaySupportAdder
2025-06-02 11:44:48 INFO: Time 0:00:03.495323 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2025-06-02 11:45:01 INFO: Time 0:00:12.972500 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2025-06

In [12]:
print(toc)

93.54334878921509


# step9: save results

In [13]:
stateVars = {}
for pop in pops.keys():
    for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
        pops[pop].write_data(f'{saveName}-{recording}.pkl')
        stateVars[pop]=pops[pop].get_data()



Getting spikes for Excitatory_Cells
|0%                          50%                         100%|
Getting v for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_exc for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_inh for Excitatory_Cells
|0%                          50%                         100%|
Getting spikes for Excitatory_Cells
|0%                          50%                         100%|
Getting v for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_exc for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_inh for Excitatory_Cells
|0%                          50%                         100%|
Getting spikes for Excitatory_Cells
|0%                          50%                         100%|
Getting v for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_exc 

# step10: recover results

In [14]:
stateVars.keys()
results = pu.recover_results(stateVars)
results.keys()

exc
v
gsyn_exc
gsyn_inh
inh
v
gsyn_exc
gsyn_inh
thalamus


dict_keys([('exc', 'v'), ('exc', 'gsyn_exc'), ('exc', 'gsyn_inh'), ('exc', 'spikes'), ('inh', 'v'), ('inh', 'gsyn_exc'), ('inh', 'gsyn_inh'), ('inh', 'spikes'), ('thalamus', 'spikes')])

# step11: postprocessing (looking at the results)

In [15]:
from pyNN.utility.plotting import Figure, Panel

Figure(
    # raster plot of the presynaptic neuron spike times
    Panel(stateVars['exc'].segments[0].spiketrains, 
          xlabel="Time/ms", xticks=True,
          yticks=True, markersize=0.2, xlim=(0, tstop)),
    title="Vogels-Abbott benchmark: excitatory cells spikes")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
### from pyNN.utility.plotting import Figure, Panel

Figure(
    # raster plot of the presynaptic neuron spike times
    Panel(stateVars['exc'].segments[0].spiketrains, xlabel="Time (ms)", xticks=True,
          yticks=True, markersize=0.2, xlim=(0, tstop)),
    title="Vogels-Abbott benchmark: inhibitory cells spikes")


In [17]:
# classic matplotlib rasterplot function

fig, ax = plt.subplots(2,1, sharex=True, figsize=(9,5))
p0=ax[0].eventplot(results['exc', 'spikes'], color='green')
p1=ax[1].eventplot(results['inh', 'spikes'], color='red')
[ax[i].set_xlabel('Time (ms)', fontsize=14) for i in [0,1]]
[ax[i].set_ylabel(feat, fontsize=14) for i, feat in enumerate(['Excitatory cells', 'Inhibitory cells'])]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0, 0.5, 'Excitatory cells'), Text(0, 0.5, 'Inhibitory cells')]

In [18]:
# check spikes

spike_rate = {
    'exc': pops['exc'].get_spike_counts(),
    'inh': pops['inh'].get_spike_counts()
}

idx=50
pop = 'exc'
print(pop + ' cell ' + str(idx) + ' :', spike_rate[pop][idx], 'spikes/' + str(tstop) + 'ms')

Getting spikes for Excitatory_Cells
|0%                          50%                         100%|
Getting spikes for Inhibitory_Cells
|0%                          50%                         100%|

exc cell 50 : 5 spikes/1000ms


In [19]:
# check conductances and membrane voltage potentials

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,3,figsize=(11,4), sharey=True, sharex=False)
ax = ax.flatten()
p=ax[0].hist(np.asarray(results['exc', 'v']).ravel(), 100, label='exc', log=False)
p=ax[1].hist(np.asarray(results['exc', 'gsyn_exc']).ravel(), 100, label='exc')
p=ax[2].hist(np.asarray(results['exc', 'gsyn_inh']).ravel(), 100, label='exc')

p=ax[0].hist(np.asarray(results['inh', 'v']).ravel(), 100, label='inh')
p=ax[1].hist(np.asarray(results['inh', 'gsyn_exc']).ravel(), 100, label='inh')
p=ax[2].hist(np.asarray(results['inh', 'gsyn_inh']).ravel(), 100, label='inh')

ax[0].set_xlim(-90,-40)
ax[1].set_xlim(0,0.2)
ax[2].set_xlim(0,0.5)

[ax[i].legend() for i in [0,1,2]]

[ax[i].set_ylabel('populations', fontsize=14) for i in [0,1,2]]
[ax[idx].set_xlabel(feat, fontsize=14) for idx, feat in enumerate(['membrane potential [mV]', 'gsyn_exc [uS]', 'gsyn_inh [uS]'])]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'membrane potential [mV]'),
 Text(0.5, 0, 'gsyn_exc [uS]'),
 Text(0.5, 0, 'gsyn_inh [uS]')]

In [20]:
# Applica a entrambe le popolazioni
pu.plot_population(results, 'exc', fileName=fileName, saveDir=savePath)
pu.plot_population(results, 'inh', fileName=fileName, saveDir=savePath)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Figura salvata in: ../outputs/VA_balance-network/exc_summary.png


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Figura salvata in: ../outputs/VA_balance-network/inh_summary.png


In [21]:
# signal processing step, statistics, information theory tools, etc


In [22]:
fs = (1/dt)*1e3
nperseg = fs/2
plt.figure(figsize=(7, 3))
for label in ['exc', 'inh']:
    sig = np.asarray(results[label, 'v']).T  # shape (300, 1000)
    f, psd = welch(sig, fs=fs, axis=0, nperseg=nperseg)
    psd_mean = np.mean(psd, axis=1)
    psd_sem = np.std(psd, axis=1) / np.sqrt(psd.shape[1])  # SEM
    #plt.plot(f, psd_mean, label=f'{label.upper()} - Media PSD', c=color)
    #plt.fill_between(f, psd_mean - psd_sem, psd_mean + psd_sem, color=color, alpha=0.3, label=f'{label.upper()} ±1 SEM')
    # --- Fit FOOOF ---
    fm = FOOOF(peak_width_limits=[1, 12], max_n_peaks=6, aperiodic_mode='fixed')
    fm.report(f, psd_mean)   
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


FOOOF WARNING: Skipping frequency == 0, as this causes a problem with fitting.

FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 1.00 <= 2.00
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

                                                                                                  
                                   FOOOF - POWER SPECTRUM MODEL                                   
                                                                                                  
                       The model was run on the frequency range 2 - 500 Hz                        
                                 Frequency Resolution is 2.00 Hz                                  
                                                                                                  
                            Aperiodic Parameters (offset, exponent):                              
                                         -1.7989, 0.0962                                          
                                                                                                  
                                       6 peaks were found:                                        
          

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

                                                                                                  
                                   FOOOF - POWER SPECTRUM MODEL                                   
                                                                                                  
                       The model was run on the frequency range 3 - 500 Hz                        
                                 Frequency Resolution is 3.33 Hz                                  
                                                                                                  
                            Aperiodic Parameters (offset, exponent):                              
                                         -2.1639, -0.0226                                         
                                                                                                  
                                       6 peaks were found:                                        
          

In [23]:
# balance computation

# step12: close simulation


In [24]:
# if you run it, you cannot reload some cells, but it is needed to restart the overall simulation

#sim.end()

## Task 1: on the self sustained acitivity conditions

- in the image below, the panel a) shows the sustained activity conditions of the network. <br>
  Try different set of weights combinations for discovering different functional network configurations

<img src="../extra/VA_EXC_BALANCE.png">


- image taken from Vogels & Abbott (J. Neurosci, 2005) https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6725859/,